In [1]:
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import cross_val_score
from keras.models import Sequential
from keras.layers import Dense
from keras.utils import np_utils

Using Theano backend.


In [2]:
from keras import optimizers

In [3]:
def create_model10a():
    model10 = Sequential()
    model10.add(Dense(7, input_dim=9, activation='relu'))
    model10.add(Dense(7, activation='relu'))
    model10.add(Dense(7, activation='relu'))
    model10.add(Dense(7, activation='relu'))
    model10.add(Dense(7, activation='relu'))
    model10.add(Dense(7, activation='relu'))
    model10.add(Dense(7, activation='relu'))
    model10.add(Dense(7, activation='relu'))
    model10.add(Dense(7, activation='relu'))
    model10.add(Dense(7, activation='relu'))
    model10.add(Dense(1, activation='sigmoid'))
    adm = optimizers.Adam(lr=0.01, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0)
    model10.compile(loss='binary_crossentropy', optimizer=adm, metrics=['accuracy'])
    return model10

In [4]:
class_weight = [{0 : 1., 1: 2},
                {0 : 1., 1: 4},
                {0 : 1., 1: 8},
                {0 : 1., 1: 16},
                {0 : 1., 1: 32},
                {0 : 1., 1: 64}]

In [5]:
epochs = [10, 20, 50, 100, 200, 500]

In [6]:
param_gridNN = dict(epochs=epochs, class_weight=class_weight )

In [7]:
from sklearn.metrics import confusion_matrix

In [8]:
modelNN = KerasClassifier(build_fn=create_model10a, verbose=0)

In [9]:
import pandas as pd
import numpy as np

In [10]:
dfb = pd.read_csv("wbc.csv")
dfb=dfb[dfb.bare_nuclei!='?']
dfb['bare_nuclei']=dfb['bare_nuclei'].astype(object).astype(int)

In [11]:
X = dfb.drop([dfb.columns[-1]], axis=1)
y = dfb.jenis.map({2: 0, 4: 1})

In [12]:
RAND_SEED_SPLIT = RAND_SEED_TUNING = 168

In [13]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y,test_size=0.33, random_state=RAND_SEED_SPLIT)

In [14]:
from sklearn.metrics import confusion_matrix

In [15]:
def cost_breast_cancer(y_true, y_pred):
    CM = confusion_matrix(y_true, y_pred)
    FN = CM[1][0]
    FP = CM[0][1]
    costBC=((-1*((228.35*FP)+(2850000*FN))))
    return costBC

In [16]:
def specificity(y_true, y_pred):
    CM = confusion_matrix(y_true, y_pred)
    TN = CM[0][0]
    FN = CM[1][0]
    TP = CM[1][1]
    FP = CM[0][1]
    nilai=(TN/(FP+TN))
    return nilai

In [17]:
from sklearn.metrics import make_scorer
my_custom_scorer=make_scorer(cost_breast_cancer, greater_is_better=True)
score_specificity=make_scorer(specificity, greater_is_better=True)

In [18]:
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
from sklearn import metrics
from sklearn.model_selection import GridSearchCV
from __future__ import print_function
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import make_scorer
from sklearn.model_selection import cross_val_score
from sklearn.metrics import roc_curve, roc_auc_score

In [19]:
print(__doc__)


scores = [my_custom_scorer]

for score in scores:
    print("# Tuning hyper-parameters for %s" % score)
    print()

    clf3 = GridSearchCV(estimator=modelNN, param_grid=param_gridNN, cv=10, 
                       scoring=my_custom_scorer)
    clf3.fit(X_train.as_matrix(), y_train)

    print("Best parameters set found on development set:")
    print()
    print(clf3.best_params_)
    print()
    print("Grid scores on development set:")
    print()
    means3 = clf3.cv_results_['mean_test_score']
    stds3 = clf3.cv_results_['std_test_score']
    for mean, std, params in zip(means3, stds3, clf3.cv_results_['params']):
        print("%0.3f (+/-%0.03f) for %r"
              % (mean, std * 2, params))
    print()

Automatically created module for IPython interactive environment
# Tuning hyper-parameters for make_scorer(cost_breast_cancer)

Best parameters set found on development set:

{'class_weight': {0: 1.0, 1: 16}, 'epochs': 10}

Grid scores on development set:

-849195.351 (+/-3630951.399) for {'class_weight': {0: 1.0, 1: 2}, 'epochs': 10}
-6591998.676 (+/-31934017.087) for {'class_weight': {0: 1.0, 1: 2}, 'epochs': 20}
-1141725.950 (+/-2792517.301) for {'class_weight': {0: 1.0, 1: 2}, 'epochs': 50}
-1996724.909 (+/-4443508.009) for {'class_weight': {0: 1.0, 1: 2}, 'epochs': 100}
-1709771.066 (+/-3786020.825) for {'class_weight': {0: 1.0, 1: 2}, 'epochs': 200}
-1135445.156 (+/-2790399.573) for {'class_weight': {0: 1.0, 1: 2}, 'epochs': 500}
-854903.022 (+/-2611035.017) for {'class_weight': {0: 1.0, 1: 4}, 'epochs': 10}
-1135376.701 (+/-2790419.102) for {'class_weight': {0: 1.0, 1: 4}, 'epochs': 20}
-1422247.598 (+/-2850268.660) for {'class_weight': {0: 1.0, 1: 4}, 'epochs': 50}
-282264.503 

In [20]:
print("Best score obtained: {0}".format(clf3.best_score_))
print("Parameters:")
for key, value in clf3.best_params_.items():
    print("\t{}: {}".format(key, value))

Best score obtained: -1007.3382932166303
Parameters:
	class_weight: {0: 1.0, 1: 16}
	epochs: 10


In [21]:
best_class_weight = {0: 1.0, 1: 16}

In [22]:
modelNN.fit(X_train.as_matrix(), y_train, epochs= 10, class_weight = best_class_weight)

In [23]:
scores4=cross_val_score(modelNN,X_train.as_matrix(),y_train,cv=10,scoring=my_custom_scorer)
print("Score cost: %0.4f (+/- %0.4f)" % (scores4.mean(), scores4.std() * 2))

Score cost: -11115228.3500 (+/- 36269855.2315)


In [24]:
scores1=cross_val_score(modelNN,X_train.as_matrix(),y_train,cv=10,scoring=score_specificity)
print("Specificity: %0.4f (+/- %0.4f)" % (scores1.mean(), scores1.std() * 2))
scores2=cross_val_score(modelNN,X_train.as_matrix(),y_train,cv=10,scoring='accuracy')
print("Accuracy: %0.4f (+/- %0.4f)" % (scores2.mean(), scores2.std() * 2))
scores3=cross_val_score(modelNN,X_train.as_matrix(),y_train,cv=10,scoring='recall')
print("Recall: %0.4f (+/- %0.4f)" % (scores3.mean(), scores3.std() * 2))


Specificity: 0.9574 (+/- 0.0661)
Accuracy: 0.8729 (+/- 0.2556)
Recall: 0.9726 (+/- 0.0696)
